### LIBRARY & MODEL IMPORTS

In [ ]:
!pip -q install nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.0/197.0 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 M

In [ ]:
import os
import cv2
import shutil
import random

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
!gdown -q 1PUFSRpCnLe9sya68CKlnEoU9aWcnthSV # Download nnUNetv2 Model
!unzip -q /content/nnUNet_results.zip # Unzip Model
!mv /content/content/nnUNet_results /content/nnUNet_results
!rm -rf /content/content

In [ ]:
# Set Environment Variables

%env nnUNet_raw = /content/nnUNet_raw/
%env nnUNet_results = /content/nnUNet_results/
%env nnUNet_preprocessed = /content/nnUNet_preprocessed/

env: nnUNet_raw=/content/nnUNet_raw/
env: nnUNet_results=/content/nnUNet_results/
env: nnUNet_preprocessed=/content/nnUNet_preprocessed/


### INPUT DATA PREPARATION

In [ ]:
# Loop Through Input
# Convert to GrayScale
# Replace .png with _0000.png

In [ ]:
!mkdir INPUT_DATA
!mkdir OUTPUT_DATA

In [ ]:
def prepare_data(input_path, output_path):
  # Converting Input Data to GrayScale
  # Naming to nnUNet Appropriate Inference Format

  CNT = 0
  for img in os.listdir(input_path):
    if img.endswith('.png'):
      CNT += 1
      image = Image.open(os.path.join(input_path, img))
      image = image.convert('L') # Convert to GrayScale
      image.save(os.path.join(output_path, img.replace('.png', '_0000.png'))) # Rename Images
  print(f'{CNT} Scans Ready for Testing')

In [ ]:
# Parse the path to the test data here:

test_data = '/content/TEST_DATA'

prepare_data(test_data, '/content/INPUT_DATA')

76 Scans Ready for Testing


### nnU-Netv2 Prediction

In [ ]:
# CUDA is required to run inference

!nnUNetv2_predict -i INPUT_DATA -o OUTPUT_DATA -d 101 -c CONFIGURATION -f 1


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/usr/local/lib/python3.10/dist-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary 

In [ ]:
# Convert images in output to 1-bit depth

for image in os.listdir('/content/OUTPUT_DATA'):
  if image.endswith('.png'):
    img = Image.open(f'/content/OUTPUT_DATA/{image}')
    img = np.array(img)
    cv2.imwrite(f'/content/OUTPUT_DATA/{image}', img, [cv2.IMWRITE_PNG_BILEVEL, 1])

In [ ]:
# Predictions are saved in OUTPUT_DATA
# Use 'plt.imshow(, cmap = 'gray')' to see masks where possible.

### HIDDEN

In [ ]:
# Randomly Move Images to TEST_DATA

CNT = 0
for root, dir, img in os.walk('/content/PRECISE_Hackathon_Dataset'):
  for image in img:
    if 'mask' not in image:
      source_path = os.path.join(root, image)
      if random.randint(0,100) < 2: # 2% chance of moving
          shutil.move(source_path, '/content/TEST_DATA')
          CNT +=1

print(f'{CNT} images moved.')


In [ ]:
!gdown -q 1g61gMabSflHdZzlQ5o6KUq5hpmIYZJI4 # Download PRECISE Hack Dataset
!unzip -q /content/PRECISE_Hackathon_Dataset.zip # Unzip Dataset
!mkdir TEST_DATA

In [ ]:
benign = os.listdir('/content/PRECISE_Hackathon_Dataset/benign')
malignant = os.listdir('/content/PRECISE_Hackathon_Dataset/malignant')
normal = os.listdir('/content/PRECISE_Hackathon_Dataset/normal')

In [ ]:
!kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset

Dataset URL: https://www.kaggle.com/datasets/aryashah2k/breast-ultrasound-images-dataset
License(s): CC0-1.0
 95% 185M/195M [00:03<00:00, 95.1MB/s]
100% 195M/195M [00:03<00:00, 64.3MB/s]


In [ ]:
!unzip -q /content/breast-ultrasound-images-dataset.zip

In [ ]:
for img in (os.listdir('/content/OUTPUT_DATA')):
  if 'png' not in img:
    print(img)

In [ ]:
combined = benign + malignant + normal

CNT = 0

for root, dir, file in os.walk('/content/Dataset_BUSI_with_GT'):
  for image in file:
    if image not in combined and 'mask' not in image:
      shutil.copy(os.path.join(root, image), '/content/TEST_DATA')
      CNT += 1

print(f'{CNT} images copied.')

76 images copied.


In [ ]:
for images in os.listdir('/content/TEST_DATA'):
  if 'mask_1' in images:
    os.rename(os.path.join('/content/TEST_DATA', images), os.path.join('/content/TEST_DATA', images.replace('mask_1', 'mask')))
  if 'mask_2' in images:
    os.rename(os.path.join('/content/TEST_DATA', images), os.path.join('/content/TEST_DATA', images.replace('mask_2', 'mask')))

In [ ]:
!zip -r /content/PRED_MASKS.zip /content/OUTPUT_DATA

  adding: content/OUTPUT_DATA/ (stored 0%)
  adding: content/OUTPUT_DATA/malignant (205).png (deflated 6%)
  adding: content/OUTPUT_DATA/benign (87).png (deflated 37%)
  adding: content/OUTPUT_DATA/normal (126).png (deflated 84%)
  adding: content/OUTPUT_DATA/benign (73).png (deflated 5%)
  adding: content/OUTPUT_DATA/benign (429).png (deflated 33%)
  adding: content/OUTPUT_DATA/malignant (206).png (deflated 8%)
  adding: content/OUTPUT_DATA/benign (9).png (deflated 12%)
  adding: content/OUTPUT_DATA/benign (6).png (deflated 23%)
  adding: content/OUTPUT_DATA/normal (131).png (deflated 82%)
  adding: content/OUTPUT_DATA/benign (61).png (deflated 28%)
  adding: content/OUTPUT_DATA/benign (67).png (deflated 6%)
  adding: content/OUTPUT_DATA/benign (433).png (deflated 31%)
  adding: content/OUTPUT_DATA/benign (72).png (deflated 13%)
  adding: content/OUTPUT_DATA/benign (66).png (deflated 26%)
  adding: content/OUTPUT_DATA/benign (82).png (deflated 24%)
  adding: content/OUTPUT_DATA/normal

In [ ]:
!rm -rf /content/TEST_DATA/*